In [ ]:
! pip install kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip features.csv.zip
!unzip train.csv.zip
!unzip test.csv.zip
!unzip sampleSubmission.csv.zip

Mounted at /content/drive
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 932MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  features.csv.zip
  inflating: features.csv            
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    


In [ ]:
!pip install neuralforecast


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
features = pd.read_csv("features.csv")
stores = pd.read_csv("stores.csv")

In [ ]:
train['Date'] = pd.to_datetime(train.Date)
smp_sub = pd.read_csv("sampleSubmission.csv")
smp_sub.head()


,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0


In [ ]:
SPLIT_DATE = pd.Timestamp('2011-11-30')

# Split the dataframe based on date
train_df = train[train["Date"] < SPLIT_DATE]
val_df = train[train["Date"] >= SPLIT_DATE]

# Separate features and target
X_train = train_df.drop(columns=["Weekly_Sales"])
X_val = val_df.drop(columns=["Weekly_Sales"])

Y_train = train_df["Weekly_Sales"]
Y_val = val_df["Weekly_Sales"]

In [ ]:
import numpy as np

# WMAE Metric Definition
def wmae(y_true, y_pred, is_holiday):
    """
    Weighted Mean Absolute Error.
    Weights = 5 if is_holiday else 1
    """
    weights = np.where(is_holiday == 1, 5, 1)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# Preprocessing

In [ ]:
import torch
import pandas as pd
from neuralforecast.models import NBEATS
from neuralforecast import NeuralForecast
from sklearn.base import BaseEstimator, TransformerMixin

class N_Beats(NBEATS):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.9)

    def set_optim(self, optimizer):
        self.optimizer = optimizer

    def set_scheduler(self, scheduler):
        self.scheduler = scheduler

    def configure_optimizers(self):
        return {
            'optimizer': self.optimizer,
            'lr_scheduler': {
                'scheduler': self.scheduler,
                'interval': 'epoch',
                'frequency': 1
            }
        }

class Preprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        x_copy = X.copy()
        if 'y' not in x_copy.columns:
            x_copy['y'] = 0
        x_copy['Date'] = pd.to_datetime(x_copy['Date'])
        return x_copy

class NBEATSModel(BaseEstimator, TransformerMixin):
    def __init__(self, models: list[NBEATS] = None):
        if models is None or len(models) == 0:
            device = 'gpu' if torch.cuda.is_available() else 'cpu'
            models = [N_Beats(
                input_size=40,
                h=48,
                max_steps=1300,
                batch_size=64,
                stack_types=['identity', 'trend', 'seasonality'],
                n_blocks=[1, 1, 1],
                random_seed=42,
                accelerator=device,
                devices=1,
                logger=True,
                enable_progress_bar=False,
                enable_model_summary=False,
            )]
        self.nf = NeuralForecast(models=models, freq='W-FRI')

    def fit(self, X, y=None):
        df = self.create_nbeats(X, y)
        df.sort_values(by=['ds'], inplace=True)
        self.nf.fit(df)
        return self

    def transform(self, X):
        preds = self.nf.predict()
        real = self.create_nbeats(X, X['y'])
        merged = pd.merge(real, preds, on=['unique_id', 'ds'], how='left')
        merged.fillna(0, inplace=True)
        return merged

    def predict(self, X=None):
        return self.transform(X)

    def create_nbeats(self, x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:
        df = pd.DataFrame()
        df["ds"] = pd.to_datetime(x["Date"])
        df["Store"] = x["Store"]
        df["Dept"] = x["Dept"]
        df["IsHoliday"] = x["IsHoliday"].astype(int)
        df["unique_id"] = df["Dept"].astype(str) + "_" + df["Store"].astype(str)
        df["y"] = y.values
        return df[["unique_id", "ds", "y", "IsHoliday"]].copy()

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tchan22 (tchan22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Hyperparameters selection

In [ ]:
import wandb
from sklearn.pipeline import Pipeline

learning_rate = [0.75, 0.65, 0.55]
h = [24, 48]
input_size = [40, 45, 50]

for lr in learning_rate:
    for h_ in h:
        for input_size_ in input_size:
            # Initialize wandb run
            wandb.init(
                project="walmart-forecasting",
                name="NBeats",
                config={
                    "learning_rate_gamma": lr,
                    "h": h_,
                    "input_size": input_size_,
                    "max_steps": 1300,
                    "batch_size": 64
                },
                reinit=True
            )

            estm = N_Beats(
                input_size=input_size_,
                h=h_,
                max_steps= 1300,
                batch_size=64,
                stack_types=(['identity', 'trend', 'seasonality']),
                n_blocks=[1, 1, 1],
                random_seed=42,
                devices=1,
                logger=False,
                enable_progress_bar=False,
                enable_model_summary=False,
            )

            estm.set_scheduler(
                torch.optim.lr_scheduler.StepLR(estm.optimizer, step_size=10, gamma=lr)
            )
            estimators = [estm]

            pipeline = Pipeline([
                ('preprocess', Preprocessor()),
                ('model', NBEATSModel(estimators))
            ])

            model = pipeline.fit(X_train, Y_train)

            # Log final training and validation loss
            try:
                wandb.log({
                    "train_loss": estm.loss_history['train'][-1],
                    "val_loss": estm.loss_history['val'][-1]
                })
            except:
                print("Could not log train/val loss")

            X_val['y'] = Y_val
            preds = model.predict(X_val)

            score = wmae(preds['y'], preds['N_Beats'], preds['IsHoliday'])

            # Log wmae score
            wandb.log({
                "wmae": score
            })

            print(f'learning rate = {lr}, input_size = {input_size_}, h = {h_}, wmae = {score}')

            wandb.finish()

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Could not log train/val loss
learning rate = 0.75, input_size = 40, h = 24, wmae = 8751.476638145288


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


wmae,▁
wmae,8751.47664


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1300` reached.


...

wmae,▁
wmae,2265.149


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Could not log train/val loss
learning rate = 0.55, input_size = 50, h = 48, wmae = 2768.9286166893557


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


wmae,▁
wmae,2768.92862


# final model selection

In [ ]:
import wandb
from sklearn.pipeline import Pipeline

batch_sizes = [64, 128]

for batch in batch_sizes:
    try:
        wandb.init(
            project="walmart-forecasting",
            name=f"NBeats-batch{batch}",
            config={
                "learning_rate_gamma": 0.65,
                "h": 48,
                "input_size": 40,
                "max_steps": 1300,
                "batch_size": batch
            },
            reinit=True
        )

        estm = N_Beats(
            input_size=40,
            h=48,
            max_steps=1300,
            batch_size=batch,
            stack_types=(['identity', 'trend', 'seasonality']),
            n_blocks=[1, 1, 1],
            random_seed=42,
            devices=1,
            logger=False,
            enable_progress_bar=False,
            enable_model_summary=False,
        )

        estm.set_scheduler(torch.optim.lr_scheduler.StepLR(estm.optimizer, step_size=12, gamma=0.65))
        estimators = [estm]

        pipeline = Pipeline([
            ('preprocess', Preprocessor()),
            ('model', NBEATSModel(estimators))
        ])

        # Fit model
        model = pipeline.fit(X_train, Y_train)

        # Log train/val loss if available
        try:
            wandb.log({
                "train_loss": estm.loss_history['train'][-1],
                "val_loss": estm.loss_history['val'][-1]
            })
        except Exception as e:
            print(f"[WARN] Could not log losses: {e}")

        # Predict and score
        X_val['y'] = Y_val
        preds = model.predict(X_val)

        score = wmae(preds['y'], preds['N_Beats'], preds['IsHoliday'])

        # Log final WMAE
        wandb.log({"wmae": score})
        print(f'[BATCH={batch}] WMAE: {score:.4f}')

    finally:
        wandb.finish()

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[WARN] Could not log losses: 'N_Beats' object has no attribute 'loss_history'
[BATCH=64] WMAE: 2136.6523


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


wmae,▁
wmae,2136.65228


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[WARN] Could not log losses: 'N_Beats' object has no attribute 'loss_history'
[BATCH=128] WMAE: 2091.3224


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


wmae,▁
wmae,2091.32244


In [ ]:
test['Date'] = pd.to_datetime(test['Date'])
preds = pipeline.predict(test)
preds

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


,unique_id,ds,y,IsHoliday,N_Beats
0,1_1,2012-11-02,0,0,0.0
1,1_1,2012-11-09,0,0,0.0
2,1_1,2012-11-16,0,0,0.0
3,1_1,2012-11-23,0,1,0.0
4,1_1,2012-11-30,0,0,0.0
...,...,...,...,...,...
115059,98_45,2013-06-28,0,0,0.0
115060,98_45,2013-07-05,0,0,0.0
115061,98_45,2013-07-12,0,0,0.0
115062,98_45,2013-07-19,0,0,0.0


In [ ]:
test

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


In [ ]:
preds = preds[['Id', 'N_Beats']]

submission = pd.merge(test[['Id']], preds, on='Id', how='left')

submission.to_csv("submission_batch128.csv", index=False)

In [ ]:
wandb.login()

True

In [ ]:
import joblib

# Task
Save the `pipeline` object to a file and upload it as a Weights & Biases artifact.

## Save the pipeline

### Subtask:
Save the trained `pipeline` object to a file using `joblib.dump`.


**Reasoning**:
Save the trained pipeline object to a file using joblib.dump.



In [ ]:
joblib.dump(pipeline, 'pipeline.pkl')

['pipeline.pkl']

## Create and log w&b artifact

### Subtask:
Create a new Weights & Biases artifact, add the saved pipeline file to it, and then log the artifact to the current W&B run.


**Reasoning**:
Create a new Weights & Biases artifact, add the saved pipeline file to it, and then log the artifact to the current W&B run.



**Reasoning**:
The previous command failed because the Weights & Biases run was finished. Re-initialize the Weights & Biases run before logging the artifact.



In [ ]:
wandb.init(project="walmart-forecasting", name="pipeline_artifact_logging", reinit=True)
artifact = wandb.Artifact('pipeline_artifact', type='model')
artifact.add_file('pipeline.pkl')
wandb.log_artifact(artifact)

<Artifact pipeline_artifact>

In [ ]:
wandb.finish()

## Summary:

### Data Analysis Key Findings

* The trained `pipeline` object was successfully saved to a file named `pipeline.pkl` using `joblib.dump()`.
* A Weights & Biases artifact named `pipeline_artifact` of type `model` was created.
* The saved `pipeline.pkl` file was successfully added to the Weights & Biases artifact.
* The artifact containing the pipeline was successfully logged to the current Weights & Biases run after re-initializing the run.

### Insights or Next Steps

* Ensure `wandb.init()` is called before attempting to log artifacts to ensure an active W&B run.
* The logged artifact can now be used to load and reuse the trained pipeline in future experiments or deployments.
